In [1]:
%load_ext autoreload
%autoreload 2

# Example of a pvsystemprofiler function


## 8/10/2021

### Notebook setup and library imports

In [2]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# PVInsight Code Imports
from solardatatools import DataHandler
from solardatatools.dataio import get_pvdaq_data
sys.path.append('..')
sys.path.append('~/github/pv-system-profiler')
from pvsystemprofiler.estimator import ConfigurationEstimator
from pvsystemprofiler.longitude_study import LongitudeStudy

### Load data table from external source

For today's example, we're loading data from NREL's PVDAQ API, which is a publically available PV generatation data set. 

In [3]:
data_frame = get_pvdaq_data(sysid=1199, year=[2015, 2016, 2017], api_key='DEMO_KEY')

CAUTION: Multiple scan rates detected!=======================] 100.0% ...queries complete in 24.8 seconds       
Scan rates (in seconds): [300, 280]
0 transitions detected.
Suggest splitting data set between:




In [4]:
data_frame.head()

,SiteID,ac_power,dc_power,inv1_ac_power,inv1_dc_current,inv1_dc_power,inv1_dc_voltage,inv1_temp,inv2_ac_power,inv2_dc_current,...,inv6_ac_power,inv6_dc_current,inv6_dc_power,inv6_dc_voltage,inv6_temp,inv7_ac_power,inv7_dc_current,inv7_dc_power,inv7_dc_voltage,inv7_temp
2015-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-01 00:05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-01 00:10:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-01 00:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-01 00:20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Running the `DataHandler` processing pipeline

In [5]:
dh = DataHandler(data_frame)

In [6]:
dh.run_pipeline(power_col='ac_power')

total time: 26.34 seconds
--------------------------------
Breakdown
--------------------------------
Preprocessing              11.25s
Cleaning                   12.71s
Filtering/Summarizing      2.38s
    Data quality           0.44s
    Clear day detect       0.30s
    Clipping detect        0.32s
    Capacity change detect 1.33s



### Site longitude and latitude downloaded from https://maps.nrel.gov/pvdaq/ PVDAQ Contributed Sites

In [7]:
latitude = 39.4856
longitude = -76.6636
tilt = 180
azimuth = 20
gmt_offset = -5

### Import sunrise_sunset method to calculate the daily solar noon

In [8]:
from pvsystemprofiler.algorithms.optimized_sunrise_sunset import get_optimized_sunrise_sunset

### Prepare input values to estimate the daily sunrise and sunset hour

### For this function, one of the input values is set to None at one time. It gives the option to calculate sunrise and sunset hours using either the raw data matrix (rdm) or the filled data matrix (fdm)

In [9]:
#fdm = None
#rdm = dh.raw_data_matrix
fdm = dh.filled_data_matrix
rdm = None

In [10]:
day_of_year = dh.day_index.dayofyear

In [11]:
days = dh.daily_flags.no_errors

### The method opt_dict returns a dictionary with the daily sunrise and sunset hours and the threshold value

In [29]:
opt_dict = get_optimized_sunrise_sunset(fdm, rdm)

In [30]:
opt_dict

{'est_sr_raw': None,
 'est_ss_raw': None,
 'meas_sr_raw': None,
 'meas_ss_raw': None,
 'thres_raw': None,
 'est_sr_f': array([6.13399208, 6.13491374, 6.13553589, ..., 6.13096545, 6.13262545,
        6.13399208]),
 'est_ss_f': array([15.95606159, 15.96324724, 15.97135561, ..., 15.94540545,
        15.95026762, 15.95606159]),
 'meas_sr_f': array([6.41666667, 6.5       , 6.75      , ..., 6.58333333, 7.66666667,
        6.58333333]),
 'meas_ss_f': array([15.83333333, 15.75      , 15.25      , ..., 15.75      ,
        15.58333333, 15.83333333]),
 'thres_f': 1e-05}

### Estimate solar noon using estimated values in the dictionary

In [31]:
sunrise = opt_dict['meas_sr_f']
sunset = opt_dict['meas_ss_f']
sunrise[np.isnan(sunrise)] = 0
sunset[np.isnan(sunset)] = 0

In [32]:
solarnoon = np.nanmean([sunrise, sunset], axis=0)

In [33]:
from pvsystemprofiler.utilities.equation_of_time import eot_da_rosa, eot_duffie

### Equation of time prosposed by Duffie

In [34]:
eot_duffie = eot_duffie(day_of_year)

### Equation of time prosposed by da Rosa

In [35]:
eot_da_rosa = eot_da_rosa(day_of_year)

### Fit longitude using previously estimated solarnoon

In [36]:
from pvsystemprofiler.algorithms.longitude.fitting import fit_longitude

In [37]:
fit_longitude(eot_duffie, solarnoon, days, gmt_offset, loss='l2')

-71.96605190107411

### Fit longitude using parameters internal to the LongitudeStudy class

In [21]:
lon_study = LongitudeStudy(data_handler=dh, gmt_offset=gmt_offset, true_value=longitude)

In [22]:
lon_study.run(verbose=False, data_matrix='filled', estimator='fit_l2', day_selection_method='all',
              eot_calculation='duffie', solar_noon_method='optimized_measurements')

### use solarnoon calculated internally

In [23]:
solarnoon2 = lon_study.solarnoon

### Use days calculated internally

In [24]:
day_of_year2 = lon_study.data_handler.day_index.dayofyear

### Use internal eout Duffie

In [25]:
eot_duffie2 = lon_study.eot_duffie

In [26]:
days2 = lon_study.days

In [27]:
fit_longitude(eot_duffie2, solarnoon2, days2, gmt_offset, loss='l2')

-71.96605190107411